Below is my attempt to scrap all ~965 (as of April 30) reviews from https://www.imdb.com/title/tt0436992/reviews (Dr Who Review) and then do sentiment analysis on them.

I was not able to figure out how to get all ~965 reviews using R so I switched to python. I save the reviews in `DrWho_reviews.csv`.

## Scrape First!

In [44]:

#Amanda ~April 30 22:40 
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.imdb.com/title/tt0436992/reviews/_ajax?paginationKey="
initial_url = "https://www.imdb.com/title/tt0436992/reviews"
ratings = []
reviews = []

def scrape_reviews(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    for review_container in soup.find_all("div", class_="review-container"):
        rating_tag = review_container.find("span", class_="rating-other-user-rating")
        rating = rating_tag.find("span").text if rating_tag else None
        ratings.append(rating)
        
        review = review_container.find("div", class_="text show-more__control").text
        reviews.append(review)
        
    next_pagination_key = soup.find("div", class_="load-more-data")["data-key"] if soup.find("div", class_="load-more-data") else None
    return next_pagination_key


next_key = scrape_reviews(initial_url)


while next_key:
    next_url = base_url + next_key
    next_key = scrape_reviews(next_url)


reviews = pd.DataFrame({"rating": ratings, "review": reviews})
reviews.to_csv("DrWho_reviews.csv", index=False)


In [45]:
#!pip install transformers
#you will need to uncomment the above to run this code block
from transformers import pipeline, AutoTokenizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
from google.colab import files
files.download("DrWho_reviews_sentiments.csv") #in case you want to keep the reviews on your machine ... no sentiment yet though!

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
review_list=reviews['review'].tolist()

In [28]:
reviews['review'].isna().sum()

0

##Sentiment Analysis below!


In [48]:
nlp = pipeline('sentiment-analysis')
reviews.dropna(subset=['review'], inplace=True)
max_length = 512

index = 0
for review in reviews['review']:
    review_chunks = [review[i:i+max_length] for i in range(0, len(review), max_length)]
    
    for chunk in review_chunks:
        result = nlp(chunk)
        label = result[0]['label']
        score = result[0]['score']
        reviews.at[index, 'sentiment_label'] = label
        reviews.at[index, 'sentiment_score'] = score
        index += 1












No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [42]:



# Create new columns for sentiment label and score
reviews['sentiment_label'] = ''
reviews['sentiment_score'] = 0.0

# Process each review chunk and update the corresponding row in the reviews dataframe
index = 0
for review in review_list:
    review_chunks = [review[i:i+max_length] for i in range(0, len(review), max_length)]
    for chunk in review_chunks:
        result = nlp(chunk)
        label = result[0]['label']
        score = result[0]['score']
        reviews.at[index, 'sentiment_label'] = label
        reviews.at[index, 'sentiment_score'] = score
        index += 1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [49]:
reviews

,rating,review,sentiment_label,sentiment_score
0,9,Seasons 1 through 10 are brilliant overall. Se...,NEGATIVE,0.998437
1,10,I cannot believe it's been back on our screens...,POSITIVE,0.998673
2,8,Doctor Who was at it's absolute peak with Chri...,POSITIVE,0.993631
3,None,"I thought nothing could kill Dr. Who, but appa...",POSITIVE,0.547901
4,10,"For as long as I can remember, I've heard abou...",POSITIVE,0.929964
...,...,...,...,...
2478,NaN,NaN,POSITIVE,0.998242
2479,NaN,NaN,POSITIVE,0.845370
2480,NaN,NaN,NEGATIVE,0.954995
2481,NaN,NaN,POSITIVE,0.937876


In [50]:
reviews = reviews.dropna(subset=['review'])

In [51]:
reviews

,rating,review,sentiment_label,sentiment_score
0,9,Seasons 1 through 10 are brilliant overall. Se...,NEGATIVE,0.998437
1,10,I cannot believe it's been back on our screens...,POSITIVE,0.998673
2,8,Doctor Who was at it's absolute peak with Chri...,POSITIVE,0.993631
3,None,"I thought nothing could kill Dr. Who, but appa...",POSITIVE,0.547901
4,10,"For as long as I can remember, I've heard abou...",POSITIVE,0.929964
...,...,...,...,...
960,None,I just wanted to say it was fantastic as the D...,POSITIVE,0.976842
961,None,Not sure if this really contains a spoiler but...,POSITIVE,0.999304
962,None,The new Doctor Who series is not available in ...,POSITIVE,0.999704
963,None,What can i say? Who is back. And I cannot be m...,POSITIVE,0.999528


In [52]:
reviews.to_csv("DrWho_reviews_sentiments_now.csv", index=False)
files.download("DrWho_reviews_sentiments_now.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>